In [1]:
import time
#import pandas as pd
import modin.pandas as mpd
import ray
ignore_reinit_error=True
ray.init()
#import polars as pl
#from datatable import dt, f, by, g, join, sort, update, ifelse

2023-03-09 18:02:44,116	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8265


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.9.13', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': 'tcp://127.0.0.1:64965', 'raylet_socket_name': 'tcp://127.0.0.1:64867', 'webui_url': '127.0.0.1:8265', 'session_dir': 'C:\\Users\\jay\\AppData\\Local\\Temp\\ray\\session_2023-03-09_18-02-42_164448_18244', 'metrics_export_port': 65435, 'gcs_address': '127.0.0.1:64835', 'address': '127.0.0.1:64835', 'node_id': '2e018927715cd6c95a800a3b73cd308aa46552201c3842ce72f219dc'})

In [2]:
file_1 = '10_million_1.csv'
file_2 = '10_million_2.csv'
num_ran = 2

In [ ]:
lib_name = 'pandas'
stats = []

for i in range (num_ran):
    stats_inner = {}
    start = time.time()
    df_1 = pd.read_csv(file_1,  engine="pyarrow", use_nullable_dtypes=True, index_col = 0)
    print(f'loading csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['loading_1'] = (time.time() - start)
    
    start = time.time()
    df_2 = pd.read_csv(file_2, engine="pyarrow", use_nullable_dtypes=True, index_col = 0)
    print(f'loading csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['loading_2'] = (time.time() - start)
    
    start = time.time()
    df_1.merge(df_2, on = 'unique_id_text' )
    print(f'merge csv on unique_id_text with {lib_name} took: {time.time() - start} seconds')
    stats_inner['merging'] = (time.time() - start)

    ## out of memory
    start=time.time()
    pd.concat([df_1, df_2])
    print(f'concat csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['concat'] = (time.time() - start)
    
    start=time.time()
    df_1.groupby('text_id').sum()
    print(f'groupby and sum by text_id with {lib_name} took: {time.time() - start} seconds ')
    stats_inner['groupby'] = (time.time() - start)
    
    start=time.time()
    df_1['new_col'] = df_1['0'].apply(round)
    print(f'apply round method with {lib_name} took {time.time() - start} seconds.')
    stats_inner['apply'] = (time.time() - start)
    
    start=time.time()
    df_1.loc[df_1['text_id'] == '9999_ID']
    print(f'.loc filter took {time.time() - start} seconds.')
    stats_inner['filtering'] = (time.time() - start)
    
    start=time.time()
    df_1.loc[df_1['text_id'] == '9999_ID', 'text_id'] = 'found it'
    print(f'.loc filter and update value took {time.time() - start} seconds.')
    stats_inner['filtering & updating'] = (time.time() - start)
    
    start=time.time()
    df_1['0_new'] = df_1['0'] * 2 + 1
    print(f'simple calculation took {time.time() - start} seconds.')
    stats_inner['column calculation'] = (time.time() - start)
    
    stats.append(stats_inner)
    
print([k for k in sorted(stats[0].keys())])
pandas_stats = [round((stats[0][k] + stats[1][k])/2,5) for k in sorted(stats[0].keys())]
print(pandas_stats)

In [ ]:
df_1 = pd.read_csv(file_1,  engine="pyarrow", index_col = 0)

In [ ]:
df_1.head()

In [ ]:
df_1['unique_id_text']

# modin

In [3]:
lib_name = 'modin'

stats = []

for i in range (num_ran):
    stats_inner = {}
    start = time.time()
    df_1 = mpd.read_csv(file_1,  index_col = 0)
    print(f'loading csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['loading_1'] = (time.time() - start)
    
    start = time.time()
    df_2 = mpd.read_csv(file_2,  index_col = 0)
    print(f'loading csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['loading_2'] = (time.time() - start)
    
#     start = time.time()
#     df_1.merge(df_2, on = 'unique_id_text' )
#     print(f'merge csv on unique_id_text with {lib_name} took: {time.time() - start} seconds')
#     stats_inner['merging'] = (time.time() - start)

    ## out of memory
    start=time.time()
    mpd.concat([df_1, df_2])
    print(f'concat csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['concat'] = (time.time() - start)
    
    start=time.time()
    df_1.groupby('text_id').sum()
    print(f'groupby and sum by text_id with {lib_name} took: {time.time() - start} seconds ')
    stats_inner['groupby'] = (time.time() - start)
    
    start=time.time()
    df_1['new_col'] = df_1['0'].apply(round)
    print(f'apply round method with {lib_name} took {time.time() - start} seconds.')
    stats_inner['apply'] = (time.time() - start)
    
    start=time.time()
    df_1.loc[df_1['text_id'] == '9999_ID']
    print(f'.loc filter took {time.time() - start} seconds.')
    stats_inner['filtering'] = (time.time() - start)
    
    start=time.time()
    df_1.loc[df_1['text_id'] == '9999_ID', 'text_id'] = 'found it'
    print(f'.loc filter and update value took {time.time() - start} seconds.')
    stats_inner['filtering & updating'] = (time.time() - start)
    
    start=time.time()
    df_1['0_new'] = df_1['0'] * 2 + 1
    print(f'simple calculation took {time.time() - start} seconds.')
    stats_inner['column calculation'] = (time.time() - start)
    
    stats.append(stats_inner)
    
print([k for k in sorted(stats[0].keys())])
modin_stats = [round((stats[0][k] + stats[1][k])/2,5) for k in sorted(stats[0].keys())]
print(modin_stats)

loading csv with modin took: 7.027432203292847 seconds
loading csv with modin took: 5.830769777297974 seconds
concat csv with modin took: 0.04370999336242676 seconds
groupby and sum by text_id with modin took: 3.1504709720611572 seconds 
apply round method with modin took 0.9100008010864258 seconds.
.loc filter took 0.6415708065032959 seconds.
.loc filter and update value took 1.4270000457763672 seconds.
simple calculation took 0.8335258960723877 seconds.
loading csv with modin took: 4.644488334655762 seconds
loading csv with modin took: 4.509963512420654 seconds
concat csv with modin took: 0.04200029373168945 seconds
groupby and sum by text_id with modin took: 2.8525197505950928 seconds 
apply round method with modin took 0.8770017623901367 seconds.
.loc filter took 0.9575190544128418 seconds.
.loc filter and update value took 1.3809998035430908 seconds.
simple calculation took 0.8525257110595703 seconds.
['apply', 'column calculation', 'concat', 'filtering', 'filtering & updating', '

# polars

In [ ]:
lib_name = 'polars'
stats = []

for i in range(num_ran):
    stats_inner = {}
    start = time.time()
    df_1 = pl.read_csv(file_1)
    print(f'loading csv with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['loading_1'] = (time.time() - start)

    start = time.time()
    df_2 = pl.read_csv(file_2)
    print(f'loading csv with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['loading_2'] = (time.time() - start)

    start=time.time()
    pl.concat([df_1, df_2])
    print(f'concat (append) csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['concat'] = (time.time() - start)

    start=time.time()
    df_1.join(df_2, on = 'unique_id_text')
    print(f'join (merge) csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['merging'] = (time.time() - start)

    start=time.time()
    df_1.groupby('text_id').sum()
    print(f'groupby and sum by text_id with {lib_name} took: {time.time() - start} seconds ')
    stats_inner['groupby'] = (time.time() - start)

    start=time.time()
    df_1['new_col'] = df_1['0'].apply(round)
    print(f'apply round method with {lib_name} took {time.time() - start} seconds.')
    stats_inner['apply'] = (time.time() - start)

    start=time.time()
    df_1[df_1['text_id'] == '9999_ID']
    print(f'filter took {time.time() - start} seconds.')
    stats_inner['filtering'] = (time.time() - start)

    start=time.time()
    df_1[df_1['text_id'] == '9999_ID', 'text_id'] = 'found it'
    print(f'filter and update value took {time.time() - start} seconds.')
    stats_inner['filtering & updating'] = (time.time() - start)

    start=time.time()
    df_1['0_2'] = df_1['0'] *2 + 1
    print(f'simple column calculation with {lib_name} took {time.time() - start} seconds.')
    stats_inner['column calculation'] = (time.time() - start)
    
    stats.append(stats_inner)

print([k for k in sorted(stats[0].keys())])
polars_stats = [round((stats[0][k] + stats[1][k])/2,5) for k in sorted(stats[0].keys())]
print(polars_stats)

In [ ]:
df_1 = pl.read_csv(file_1)

In [ ]:
df_1.head()

# datatable

In [ ]:
lib_name = 'datatable'
stats = []

for i in range(num_ran):
    stats_inner = {}
    start = time.time()
    df_1 = dt.fread(file_1)
    print(f'loading csv with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['loading_1'] = (time.time() - start)
    
    start = time.time()
    df_2 = dt.fread(file_2)
    print(f'loading csv with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['loading_2'] = (time.time() - start)

    start = time.time()
    df_2 = dt.rbind(df_1, df_2)
    print(f'vertically concat csv with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['concat'] = (time.time() - start)
    
    df_3 = df_1[:,0:10]
    start = time.time()
    df_3[:,dt.sum(f[:]), by('text_id')]
    print(f'groupby and sum csv with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['groupby'] = (time.time() - start)
    
    start = time.time()
    df_1['0_2'] = f[1] * 2 +1
    print(f'simple column calculation with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['column calculation'] = (time.time() - start)
    
    start = time.time()
    df_1[f.text_id == '9999_ID',:]
    print(f'filtering with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['filtering'] = (time.time() - start)
    
    start = time.time()
    df_1[f.text_id == '9999_ID','text_id'] = 'found it'
    print(f'filtering and update value with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['filtering & updating'] = (time.time() - start)
    
    stats_inner['merge'] = 0
    stats.append(stats_inner)

print([k for k in sorted(stats[0].keys())])
datatable_stats = [round((stats[0][k] + stats[1][k])/2,5) for k in sorted(stats[0].keys())]
print(datatable_stats)

In [ ]:
df_1.head()

In [ ]:
df_1

In [ ]:
%time test('pandas')

In [ ]:
%time test('modin')

In [ ]:
import polars as pl

In [ ]:
df1 = pl.read_csv(file_1)
df2 = pl.read_csv(file_2)
df3 = pl.concat([df1, df2])

In [ ]:
df3.shape

In [ ]:
df4 = df1.join(df2, on ='unique_id_text')

In [ ]:
df4.shape

In [ ]:
df4.head()

In [ ]:
df_1.groupby('text_id').sum()